In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [15]:
# notebook imports

import os
import time
import copy
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC as SVM
from sklearn.ensemble import RandomForestClassifier as RF

import warnings
warnings.filterwarnings('ignore')

In [4]:
CSV_PATH = './resnet50_FC_512_features_with_labels.csv' 

In [5]:
# loading the features extracted previously

ftrs_df = pd.read_csv('./resnet50_FC_512_features_with_labels.csv')
ftrs_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,label
0,0.085607,0.021805,0.441109,-0.286769,0.513648,-0.185310,-0.292494,-0.433075,-0.010477,-0.379597,-0.464587,-0.311266,-0.397218,-0.035729,-0.184875,0.005529,-0.485860,-0.084951,1.079289,0.234766,-0.607950,-0.416270,0.270703,0.078726,3.390669,1.175128,1.639136,0.919151,0.297039,1.392895,-0.078611,-0.245920,-0.380388,-1.181374,0.501518,-0.359335,0.949465,-0.596463,0.978446,0.250207,...,-0.673682,0.134089,-0.830013,-0.270452,0.274271,-0.492283,1.312896,0.623881,-0.427828,-0.434984,-0.211153,0.655323,0.089081,2.196209,-0.643546,0.143208,1.332753,-0.046068,0.817310,0.190408,-0.307006,0.571560,0.143442,1.586519,0.291949,-0.515978,1.637968,1.365301,0.992635,1.900833,-0.517263,0.046018,-0.363306,-0.605235,-0.088705,0.874085,-0.474211,-0.151722,-0.569436,1
1,-0.133243,-0.964772,-0.479122,-0.388593,2.520442,-0.078540,1.397684,-0.962195,-0.197336,-0.283352,-0.368340,-0.348090,-0.860821,0.803020,-0.532486,-0.219943,-0.567403,-0.213607,1.202563,1.007898,1.166395,-0.908014,1.859558,1.871797,2.570776,1.382042,2.295932,0.721915,1.490825,0.843888,-0.023431,0.762509,-0.507451,0.378494,0.872305,-0.252291,2.568783,-1.072574,-0.142558,0.200490,...,-0.657437,0.102203,-0.975201,-0.341460,1.338218,-0.867677,1.562890,-0.927345,-0.620522,-0.262975,-0.594635,2.326258,-0.029713,1.235405,-1.075083,-0.299171,1.695334,2.343036,0.426081,0.945150,-0.855601,0.465321,0.578151,2.881992,0.689831,-0.723939,0.376424,1.517536,-0.131685,0.042126,-0.536044,-0.029770,-0.425078,-0.667606,-0.453189,2.844271,-0.952448,1.436085,1.016485,3
2,0.083105,-0.696422,-0.796738,-0.497731,1.963784,-0.284097,3.261419,-0.800969,-0.506469,-0.441910,0.079485,-0.463492,-0.471498,0.799019,-0.376445,-0.360156,-0.686314,-0.172683,1.956364,-0.891847,3.087634,-0.818431,0.335942,0.611970,0.758249,1.121906,0.802602,0.901728,3.559395,-1.312710,0.931835,2.984989,-0.468388,1.456828,0.267345,-0.318779,3.901358,-0.609226,-0.471357,1.560984,...,-0.486394,0.011836,-0.607265,-0.756973,0.369215,-0.849208,2.028156,0.119567,-0.803319,-0.465819,0.024541,0.522723,-0.393713,0.456446,-1.160259,-0.269853,1.397479,0.577923,-0.016859,1.445956,-0.849506,2.242668,1.023136,1.804294,0.145638,-0.650150,1.232863,2.362357,0.294656,0.325273,-0.361375,0.469979,-0.755104,-0.431179,0.206783,2.577395,-0.640026,0.363521,0.624068,0
3,-0.071460,-0.553484,0.029384,-0.145030,-0.509019,-0.443575,2.058645,-0.614785,-0.025296,-0.196204,1.508216,-0.386372,-0.322134,0.093845,-0.164296,-0.202908,-0.675448,1.987206,0.743067,-0.112193,1.952012,-0.279601,1.317924,1.228551,-0.335249,-0.084516,0.756525,1.279888,0.214162,-0.109619,1.804547,1.737462,-0.252732,1.941269,-0.171081,-0.347327,-0.324948,-0.587768,0.029561,-0.269150,...,1.275778,0.271461,2.004199,-0.408011,1.705920,-0.323414,-0.148019,0.805807,-0.330228,-0.294776,0.718698,0.251789,0.782802,-0.270511,-0.356032,-0.055300,0.247848,1.221817,-0.020865,1.573320,0.149294,-0.046459,0.353816,-0.122537,-0.027578,-0.480010,-0.243383,-0.362624,0.525934,-0.071147,-0.662112,-0.019017,-0.365846,-0.136268,0.428925,-0.099518,-0.303276,2.082908,0.965919,2
4,0.197739,-0.731074,-0.256347,-1.116051,-1.104760,0.012868,0.499145,-0.610310,-0.062483,-0.172752,-0.808338,-0.300046,-1.091365,-0.175848,-0.590491,-0.383289,-0.950313,1.537892,2.285882,-0.894574,1.005046,-0.548572,0.915498,2.209650,4.332871,1.777773,3.032871,2.438895,0.461631,1.467396,0.488518,0.534241,-0.472278,0.219244,0.072697,-0.203950,-0.583803,-0.649506,1.389083,-0.124601,...,-0.562442,0.392000,0.149472,-0.060157,2.243619,-0.548696,1.862436,0.320670,-0.460364,-0.493218,-1.026154,0.948146,1.560519,2.732547,-1.321941,-0.210493,2.269151,1.818774,-0.473236,2.870932,-1.494021,-0.124955,0.283038,2.440798,1.405221,-1.211263,1.504962,0.168726,0.873936,2.443866,-0.567604,-0.187

In [6]:
X = ftrs_df.iloc[:,:-1]
y = ftrs_df['label']

X = np.array(X)
y = np.array(y)

print(X.shape,y.shape)

(4323, 512) (4323,)


In [7]:
# model class for each Agent i.e. chromosome in GA

class Agent:
    # class whose object holds a numpy array and its corresponding fitness
    def __init__(self,agent,fitness):
        self.agent = agent
        self.fitness = fitness

In [19]:
# utility functions for GA

def get_selected_features(X, agent, num_ftrs_sel, agent_idx, call=1):  # getting selected feature set from the FS vector 
    X_1 = np.zeros((X.shape[0],num_ftrs_sel))
    row = 0
    for i in range(agent.shape[0]):
        if agent[i] == 1:
            X_1[:,row] = X[:,i]
            row += 1

    if call == 1 : # print FS 
        print(f'Nos of features selected for Chromosome {agent_idx+1} is: {X_1.shape[1]}')

    return X_1


def most_fit_agents(agents, gen):    # sorting an array of Agent objects by their fitness values and returning the top 50% of them

    n_fit = len(agents)//2
    agents.sort(key=lambda obj:obj.fitness,reverse=True) # sort by fitness
    print(f'\n==========> fitness of most fit chromosome of generation {gen+1} is {(agents[0].fitness):.6f}')
    agents = agents[0:n_fit] # top 50% 
    return agents


def crossover_two_agents(agent1,agent2,crossover_point=0.5):    # utlity function to perform crossover between 2 agent vectors

    child1 = agent1.copy()
    child2 = agent2.copy()
    n_len = len(agent1)

    start = int(n_len*crossover_point)
    for i in range(start,n_len):
        child1[i] = agent2[i]
        child2[i] = agent1[i]

    return child1,child2 


def mutate_agent(agent, prob_mut=0.4):  # perform mutation on agent with probab. of mutation = prob_mut

    n_len = len(agent)
    for i in range(n_len):
        if np.random.rand() < prob_mut:
            agent[i] = 1 - agent[i]
    return agent


def initialize_population(num_agents,num_features):     # initialize matrix of 0/1 randomly

    # define min and max number of features
    min_features = int(0.2 * num_features)
    max_features = int(0.5 * num_features)

    # initialize the agents with zeros
    init_population = np.zeros((num_agents, num_features))

    # select random features for each agent
    for agent_no in range(num_agents):

        # find random indices
        cur_count = np.random.randint(min_features, max_features)
        temp_vec = np.random.rand(1, num_features)
        temp_idx = np.argsort(temp_vec)[0][0:cur_count]

        # select the features with the ranom indices
        init_population[agent_no][temp_idx] = 1   

    return init_population
    

def get_fitness(X, y, original_ftrs=512, weight_acc=0.98):   # fitness of a selected set of features
    
    X = np.array(X)
    num_features = X.shape[1]
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=None,shuffle=False)

    clf = SVM() # learning algorithm for wrapper-based FS
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    
    eff_feat = (original_ftrs - num_features)/original_ftrs

    fitness = weight_acc * acc + (1-weight_acc) * eff_feat

    return fitness

In [16]:
num_generations = 20 # nos of generations of GA
num_agents = 20 # size of population
num_features = X.shape[1] # original number of features

In [17]:
best_agent_per_generation = np.zeros((num_agents,num_features)) # keeps track of most fit chromosome per generation

init_population = initialize_population(num_agents,num_features) # initial population of chromosomes

np.random.seed(random.randint(0,512)) # setting random seed

for gen in range(num_generations):

    print(f'======== Generation {gen+1} =========')

    # array of Agent(agent,fitness)
    agent_fitness = []

    for agent_idx, agent in enumerate(init_population):

        # get selected feature set from agent vector
        X1 = get_selected_features(X, agent, int(np.sum(agent)), agent_idx)

        # get fitness of that agent 
        fitness = get_fitness(X1, y, num_features)
        
        # put Agent(agent,fitness) into the array
        agent_fitness.append(Agent(agent, fitness))

    # get most fit agents of current generation
    most_fit = most_fit_agents(agent_fitness, gen)

    # convert the most fit agents to numpy array
    most_fit_np = np.zeros((len(most_fit), num_features))

    for i,obj in enumerate(most_fit):
        agent = np.array(obj.agent)
        most_fit_np[i] = agent.copy()

    # get offsprings of the most fit parents
    offspring = np.zeros((num_agents//2,num_features))

    for i in range(0,(num_agents//2 - 1), 2):

        # perform crossover between pair of parents
        child1,child2 = crossover_two_agents(most_fit_np[i],most_fit_np[i+1])

        # put them into array
        offspring[i] = child1
        offspring[i+1] = child2

    # concatenate the parents and offsprings
    most_fit_np = np.append(most_fit_np,offspring,axis=0)

    # store most fit agent
    best_agent_per_generation[gen] = most_fit_np[0].copy()

    # mutation on every agent
    for i in range(most_fit_np.shape[0]):
        most_fit_np[i] = mutate_agent(most_fit_np[i])

    # set population to the most_fit_np array
    assert init_population.shape == most_fit_np.shape

    init_population = most_fit_np.copy()

    print('---------------------------------------\n\n')


======== Generation 1 =========
Nos of features selected for Chromosome 1 is: 253
Nos of features selected for Chromosome 2 is: 169
Nos of features selected for Chromosome 3 is: 270
Nos of features selected for Chromosome 4 is: 170
Nos of features selected for Chromosome 5 is: 196
Nos of features selected for Chromosome 6 is: 286
Nos of features selected for Chromosome 7 is: 218
Nos of features selected for Chromosome 8 is: 247
Nos of features selected for Chromosome 9 is: 261
Nos of features selected for Chromosome 10 is: 237
Nos of features selected for Chromosome 11 is: 219
Nos of features selected for Chromosome 12 is: 302
Nos of features selected for Chromosome 13 is: 257
Nos of features selected for Chromosome 14 is: 248
Nos of features selected for Chromosome 15 is: 293
Nos of features selected for Chromosome 16 is: 273
Nos of features selected for Chromosome 17 is: 190
Nos of features selected for Chromosome 18 is: 197
Nos of features selected for Chromosome 19 is: 241
Nos of f

In [26]:
# get best FS vector out of all generations

max_acc_agent = np.zeros(best_agent_per_generation[0].shape)
max_acc = 0

model = SVM() 

for gen,best_agent in enumerate(best_agent_per_generation):
    
    X1 = get_selected_features(X, best_agent, int(np.sum(best_agent)), agent_idx=0, call=0)

    if X1.shape[1] > 0:
        X_train,X_test,y_train,y_test = train_test_split(X1, y, test_size=0.2, stratify=None, shuffle=False)

        # classification test
        model.fit(X_train,y_train)
        acc = model.score(X_test,y_test)

        if acc >= max_acc:
            max_acc = acc
            max_acc_agent = best_agent.copy()

print(f'most accurate FS has {int(np.sum(max_acc_agent))} features')

most accurate FS has 235 features


In [30]:
def validate_FS(best_agent, clf='SVM'):

    print(f'The FS has {int(np.sum(best_agent))} features')
    
    model = SVM() # default classifier

    if clf.upper() == 'KNN':
        model = KNN()
    elif clf.upper() == 'RF':
        model = RF()
        
    X1 = get_selected_features(X, best_agent, int(np.sum(best_agent)), agent_idx=0, call=0)

    if X1.shape[1] > 0:
        X_train,X_test,y_train,y_test = train_test_split(X1, y, test_size=0.2, stratify=None, shuffle=False)

    # classification test
    model.fit(X_train,y_train)
    acc = model.score(X_test,y_test)

    print(f'accuracy of this FS by {clf} is {(100.0*acc):.4f}')

In [31]:
best_agent = max_acc_agent.copy()
validate_FS(best_agent,clf='SVM')

The FS has 235 features
accuracy of this FS by SVM is 95.2601


In [32]:
best_agent = max_acc_agent.copy()
validate_FS(best_agent,clf='KNN')

The FS has 235 features
accuracy of this FS by KNN is 94.5665


In [33]:
best_agent = max_acc_agent.copy()
validate_FS(best_agent,clf='RF')

The FS has 235 features
accuracy of this FS by RF is 95.0289
